In [ ]:
class Genome():
	def __init__(self, size, fitness, genes):
		self.size = size
		self.fitness = fitness
		self.genes = genes

	def __str__(self):
		return "Genome: size = {}, fitness = {}, genes = {}".format(self.size, self.fitness, self.genes)

	def __repr__(self):
		return self.__str__()

class TrainGenomes():
	def __init__(self, population_size, mutation_rate, crossover_rate, elitism_rate, max_generations, max_no_improvement, max_time, train_data, test_data, train_labels, test_labels, model):
		self.population_size = population_size
		self.mutation_rate = mutation_rate
		self.crossover_rate = crossover_rate
		self.elitism_rate = elitism_rate
		self.max_generations = max_generations
		self.max_no_improvement = max_no_improvement
		self.max_time = max_time
		self.train_data = train_data
		self.test_data = test_data
		self.train_labels = train_labels
		self.test_labels = test_labels
		self.model = model

	def train(self):
		# Initialize the population
		population = self.initialize_population(self.population_size, self.train_data.shape[1])
		# Evaluate the population
		population = self.evaluate_population(population)
		# Sort the population by fitness
		population.sort(key=lambda x: x.fitness, reverse=True)
		# Keep track of the best genome
		best_genome = population[0]
		# Keep track of the number of generations with no improvement
		no_improvement = 0
		# Keep track of the number of generations
		generations = 0
		# Keep track of the time
		start_time = time.time()
		# Keep track of the best fitness
	
	def initialize_population(self, population_size, genome_size):
		population = []
		for i in range(population_size):
			genes = np.random.randint(2, size=genome_size)
			population.append(Genome(genome_size, 0, genes))
		return population
	
	def evaluate_population(self, population):
		for genome in population:
			self.model.fit(self.train_data, self.train_labels, epochs=1, verbose=0)
			genome.fitness = self.model.evaluate(self.test_data, self.test_labels, verbose=0)[1]
		return population
	
	def select_parents(self, population):
		# Select the parents using tournament selection
		parent1 = self.tournament_selection(population)
		parent2 = self.tournament_selection(population)
		return parent1, parent2
	
	def tournament_selection(self, population):
		# Select two random genomes
		genome1 = population[np.random.randint(len(population))]
		genome2 = population[np.random.randint(len(population))]
		# Return the fitter genome
		return genome1 if genome1.fitness > genome2.fitness else genome2
	
	def crossover(self, parent1, parent2):
		# Select a crossover point
		crossover_point = np.random.randint(1, parent1.size)
		# Create the offspring
		offspring1 = Genome(parent1.size, 0, np.concatenate((parent1.genes[:crossover_point], parent2.genes[crossover_point:])))
		offspring2 = Genome(parent1.size, 0, np.concatenate((parent2.genes[:crossover_point], parent1.genes[crossover_point:])))
		return offspring1, offspring2

	def mutate(self, genome):
		# Select a mutation point
		mutation_point = np.random.randint(genome.size)
		# Mutate the genome
		genome.genes[mutation_point] = 1 if genome.genes[mutation_point] == 0 else 0
		return genome
	
